In [ ]:
import pandas as pd
import numpy as np

# filename variable(s)
raw_data_observations = 'observations.csv'
raw_data_species_info = 'species_info.csv'

#-----Loading Data------
def sp_load_df(file):
    try:
        df = pd.read_csv(file)
        print('DataFrame loaded')
        return df
    except:
        print('DataFrame did not load')
    
#------Inspect------
def summarize(df):
    df = df.copy()
    missing_data = (df.isna().mean() *100).round(2).to_dict()
    print(f'\n----Column Titles: \n{list(df.columns)}\n\n')
    print('----Missing Percentages:\n')
    for key, value in missing_data.items():
        print(f'{key}, {value}% missing')
    print('\n----Data Types:\n\n', df.dtypes, '\n\n')
    print(df.describe(include='all'), '\n\n')
    print(df.info())

def col_duplicates(df, col):
    df.copy()
    count = df[col].value_counts()
    count_df = count.reset_index()
    duplicates_df = count_df[count_df['count'] > 1]
    print(f'Duplicates Dataframe: length = {len(duplicates_df)}')
    return duplicates_df

# Load, Inspect, Clean, Organize 

## 1. `observations.csv`

`df = sp_load_df(raw_data_observations)`

In [ ]:
df = sp_load_df(raw_data_observations)

In [ ]:
summarize(df)

In [ ]:
# count duplicates in scientific_name
sci_name_dupl = col_duplicates(df, 'scientific_name')

In [ ]:
sci_name_dupl.head()

In [ ]:
summarize(sci_name_dupl)

In [ ]:
# groupby approach to sci_names for duplicates
df_sci_grouped = df.groupby(['scientific_name','park_name'], as_index=False).observations.sum().copy()

In [ ]:
df_sci_grouped.head()

In [ ]:
# Check to make sure observations count didn't change
np.sum(df_sci_grouped.observations) == np.sum(df.observations)

## 2. `species_info.csv`

`df1 = sp_load_df(raw_data_species_info)`

In [ ]:
df1 = sp_load_df(raw_data_species_info)

In [ ]:
summarize(df1)

In [ ]:
df1.sort_values(by=['conservation_status', 'scientific_name'], ascending=[True, True]).to_csv('species_info_sort.csv')

In [ ]:
spc_sci_name_dup = col_duplicates(df1, 'scientific_name')

In [ ]:
spc_sci_name_dup.head()

In [ ]:
#Prepare species for merge into observations df

# Canis lupus - grey wolf is listed as 'in recovery' and 'endangered' on duplicates
# Below status and pick_status is designed to handle instances by selecting the more severe of the two

status_priority = {
    "Endangered": 4,
    "Threatened": 3,
    "Species of Concern": 2,
    "In Recovery": 1,
    "No Concern": 0,
}

def pick_status(col: pd.Series) -> str:
    # choose the single highest-priority label present for this species
    # unknown labels fall back to lowest priority (-1)
    return max(col, key=lambda x: status_priority.get(x, -1))

spc_agg = (df1
    .groupby('scientific_name', as_index=False)
    .agg({
        'common_names': lambda x: ', '.join(sorted(set(x))),
        'category': lambda x: ', '.join(sorted(set(x))),
        'conservation_status': pick_status
    })
)

In [ ]:
spc_agg.conservation_status = spc_agg.conservation_status.fillna('No Concern')

In [ ]:
spc_agg.head()

In [ ]:
# Check for sci name dups to ensure 1:1 for merge
dup_spc_agg = col_duplicates(spc_agg, 'scientific_name')

## Merge DF's on `scientific_name`


In [ ]:
df_df1_merged = df_sci_grouped.merge(
    spc_agg,
    on='scientific_name',
    how='left',
    copy=True
)

In [ ]:
df_df1_merged.head()

In [ ]:
# confirm no change to obs count for original df)
np.sum(df.observations) == np.sum(df_df1_merged.observations)

# Column Variable Modifications 

Cleaned and Merged DF = `df_df1_merged` » `df2`

In [ ]:
df2 = df_df1_merged
unord_cat_cols = ['scientific_name', 'park_name', 'category']


In [ ]:
def col_conv(df, cols):
    for col in df[cols]:
        df[col] = pd.Categorical(df[col], ordered=False) # seem to need to use the [] for col notation

In [ ]:
col_conv(df2, unord_cat_cols)

In [ ]:
# ordered categorical approach for conservation_status

df2.conservation_status = pd.Categorical(df2.conservation_status, ((list(status_priority.keys()))[::-1]), ordered=True)
    


In [ ]:
summarize(df2)

In [ ]:
print(df2.conservation_status.unique())

# Final DF for write to feather = `df2`

In [ ]:
# df2.to_feather('df_merged.feather')